In [1]:
import torch

######################################
#### Networks imports ################
import torch.nn as nn
import torch.nn.functional as F

#####################################
######### Optimizer #################
import torch.optim as optim

#############Dataset#########################
from torch.utils.data import Dataset, DataLoader

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")  

In [3]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()

In [4]:
client = Client(cluster)

In [43]:
from torch.utils.data import Dataset, DataLoader
from glob import glob
import os
import dask.array as da
import numpy as np

class StockDataset(Dataset):

    def __init__(self, datasetFolder, transforms=None):
        self.datasetFolder = datasetFolder
        self.data = self.get_data(datasetFolder)
        self.target = self.get_target(datasetFolder)
        self.length = len(self.target)
        self.transforms = transforms
    
    def get_data(self, datasetFolder):
        dsets = [da.from_array(np.load(fname)) for fname in glob(os.path.join('data', 'NPY', '*-data.npy'))]
        data = da.concatenate(dsets, axis=0)
        return data
    
    def get_target(self, datasetFolder):
        dsets = [da.from_array(np.load(fname)) for fname in glob(os.path.join('data', 'NPY', '*-target.npy'))]
        target = da.concatenate(dsets, axis=0)
        return target
    
    
    def __getitem__(self, index):
        # get data and label
        data = self.data[index].compute()
        target = self.target[index].compute()

        if self.transforms is not None:

            data = self.transforms(data)
            
        data = torch.unsqueeze(torch.from_numpy(data), dim=0)
        target = torch.from_numpy(target)   
        return data, target
    
    def __len__(self):
        return self.length

In [ ]:
class Net(nn.Module):

    def __init__(self):
        ########### define layers ################

    def forward(self, x):
        ####### Calculate output from input ##########
        return x


In [ ]:
####Loading data
batch_size = 1
epoch_num = 10
stock_dataset = StockDataset(datasetFolder, transforms=transforms)
stock_loader = DataLoader(stock_dataset, batch_size=batch_size, shuffle=True)
net = Net()
###Optimizer
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
###Criterion
criterion = nn.MSELoss()

for epoch in range(epoch_num):
    # in your training loop:
    running_loss = 0.0
    for i, data in enumerate(stock_loader, 0):
    
    # get the inputs; data is a list of [inputs, labels]*******depends on dataloader definition*****
    inputs, targets = data
    optimizer.zero_grad()   # zero the gradient buffers
    
    # forward + backward + optimize
    output = net(inputs)
    loss = criterion(output, targets)
    loss.backward()
    optimizer.step()    # Does the update
    
    # print statistics
    running_loss += loss.item()
    if i % 2000 == 1999:    # print every 2000 mini-batches
        print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss / 2000))
        running_loss = 0.0